In [8]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [28]:
# pull 990 xmls
metadata_df = pd.read_csv('output/metadata.csv', header=0)
rows = []

for xmlid in metadata_df['OBJECT_ID']:
    # pull from S3
    url = 'https://s3.amazonaws.com/irs-form-990/%d_public.xml' % xmlid
    print url
    response = requests.get(url)
    
    # parse to soup
    soup = BeautifulSoup(response.text[3:], 'xml')
    
    # get org data
    ein = soup.find('EIN').contents[0]
    try:
        org_name = soup.find('Filer').Name.BusinessNameLine1.contents[0]
    except AttributeError:
        try:
            org_name = soup.find('Filer').BusinessName.BusinessNameLine1.contents[0]
        except AttributeError:
            org_name = soup.find('Filer').BusinessName.BusinessNameLine1Txt.contents[0]
    try:
        city = soup.find('Filer').USAddress.City.contents[0]
    except AttributeError:
        city = soup.find('Filer').USAddress.CityNm.contents[0]
    try:
        state = soup.find('Filer').USAddress.State.contents[0]
    except AttributeError:
        state = soup.find('Filer').USAddress.StateAbbreviationCd.contents[0]
    try:
        tax_year = soup.find('TaxYear').contents[0]
    except AttributeError:
        tax_year = soup.find('TaxYr').contents[0]
    try:
        submission_time = soup.find('Timestamp').contents[0]
    except AttributeError:
        submission_time = soup.find('ReturnTs').contents[0]        
    try:
        year_formed = soup.find('YearFormation').contents[0]
    except AttributeError:
        try:
            year_formed = soup.find('FormationYr').contents[0]
        except:
            print 'year formed'
            year_formed = None
    try:
        n_employees = soup.find('TotalNbrEmployees').contents[0]
    except AttributeError:
        try:
            n_employees = soup.find('TotalEmployeeCnt').contents[0]
        except:
            print 'n employees'
            n_employees = None

    # get revenue
    total_revenue = None
    revenue_fields = ['TotalRevenueCurrentYear', 'TotalRevenue', 'TotalRevenueAmt',
                      'TotalRevenueAndExpenses', 'CYTotalRevenueAmt', 'TotalRevAndExpnssAmt']
    for revenue_field in revenue_fields:
        try:
            total_revenue = soup.find(revenue_field).contents[0]
            break
        except AttributeError:
            pass
    if total_revenue in [None, '\n']:
        break
        
    base_row = [
        org_name, city, state, ein, url,
        tax_year, year_formed, submission_time,
        n_employees, total_revenue,
    ]

    # get compensation data
    n_people_found = 0
    for employee in soup.find_all('Form990PartVIISectionA'):
        try:
            title = employee.Title.contents[0]
            comp = employee.ReportableCompFromOrganization.contents[0]
        except AttributeError:
            continue
        rows.append(base_row + [title, comp])
        n_people_found += 1
    for employee in soup.find_all('Form990PartVIISectionAGrp'):
        try:
            title = employee.TitleTxt.contents[0]
            comp = employee.ReportableCompFromOrgAmt.contents[0]
        except AttributeError:
            continue
        rows.append(base_row + [title, comp])
        n_people_found += 1
    for employee in soup.find_all('OfficerDirTrstKeyEmplGrp') + soup.find_all('OfficerDirectorTrusteeEmplGrp'):
        try:
            title = employee.TitleTxt.contents[0]
            comp = employee.CompensationAmt.contents[0]
        except AttributeError:
            continue
        rows.append(base_row + [title, comp])
        n_people_found += 1
    for employee in soup.find_all('OfficerDirectorTrusteeKeyEmpl') + soup.find_all('OfcrDirTrusteesOrKeyEmployee') + soup.find_all('CompensationOfHighestPaidEmpl'):
        try:
            title = employee.Title.contents[0]
            comp = employee.Compensation.contents[0]
        except AttributeError:
            continue
        rows.append(base_row + [title, comp])
        n_people_found += 1

    print org_name, n_people_found

df = pd.DataFrame(rows)

https://s3.amazonaws.com/irs-form-990/201120429349301812_public.xml
The Center to Promote Healthcare Access Inc 11
https://s3.amazonaws.com/irs-form-990/201102279349304550_public.xml
CENTER FOR STUDENT OPPORTUNITY INC 3
https://s3.amazonaws.com/irs-form-990/201132989349300003_public.xml
CLASSROOM INC 15
https://s3.amazonaws.com/irs-form-990/201103139349300945_public.xml
CODE FOR AMERICA LABS INC 6
https://s3.amazonaws.com/irs-form-990/201143199349310864_public.xml
D-REV DESIGN FOR THE OTHER 90 8
https://s3.amazonaws.com/irs-form-990/201122989349300912_public.xml
FAMILIES EMPOWERED 2
https://s3.amazonaws.com/irs-form-990/201100329349200010_public.xml
year formed
n employees
THE FREECYCLE NETWORK 3
https://s3.amazonaws.com/irs-form-990/201133189349303268_public.xml
HARMONY INSTITUTE INC 3
https://s3.amazonaws.com/irs-form-990/201133199349101553_public.xml
year formed
n employees
HopeLab Foundation Inc 9
https://s3.amazonaws.com/irs-form-990/201113199349301711_public.xml
IN OUR BACKYARDS 

In [29]:
# label and output
df.columns = [
    'org_name', 'city', 'state', 'ein', 'url',
    'tax_year', 'year_formed', 'submission_time',
    'n_employees', 'total_revenue',
    'title', 'comp',
]
df.to_csv('output/comp_data_raw.csv')

In [30]:
# full list of job titles
print df['title'].unique()

[u'Executive Director' u'Dir of App Solutions' u'director'
 u'Product Manager' u'asst dir of app solution' u'Dir Fin & Admin Thru 4/6'
 u'Exec Director/CEO' u'Board Member' u'CHAIRMAN' u'TREASURER' u'SECRETARY'
 u'DIRECTOR' u'PRESIDENT' u'VP AND COO' u'VP OF CURRICULUM & ASSESSMENT'
 u'DIRECTOR OF FINANCE' u'SENIOR DIRECTOR OF BUSINESS DEVELOPMENT'
 u'Director' u'Executive Direc' u'CHAIRMAN, TREASURER'
 u'CHIEF EXECUTIVE OFFICER' u'Founder' u'Board Chair' u'VICE CHAIR'
 u'CHAIRPERSON' u'VICE PRESIDENT' u'President & CEO' u'Dir of Prod Dev'
 u'VP Impact & Adm' u'VP Staff Dev' u'VP Strat. P/S' u'VP Research'
 u'COFOUNDER & EXECUTIVE DIRECTOR' u'COFOUNDER & CHIEF OPERATING OFFICER'
 u'COFOUNDER, DIRECTOR' u'CHAIR, BOARD OF DIRECTOR'
 u'SECRETARY, BOARD OF DIRECTORS' u'TRESURER, BOARD OF DIRECTORS'
 u'BOARD OF DIRECTORS' u'Trustee' u'President' u'Secretary' u'Treasurer'
 u'TRUSTEE' u'PRESIDENT &' u'VICE PRES' u'CHAIRMAN/CEO' u'CFO'
 u'VP AFRICA MKTG & OPERATIONS' u'VP OF LEARNING' u'VICE-P

In [31]:
# job title regex

# CEO
ceo_label = 'ceo'
ceo_regex = ['exec', 'ceo', 'e\.d\.', 'ed$']
ceo_antiregex = []

# CTO
cto_label = 'cto'
cto_regex = ['tech[n. ]', '^cto', 'software', 'engin', 'product[s ]', 'product$',
             'app', 'developer', 'archit', 'prod dev']
cto_antiregex = ['regional', 'service']

# COO
coo_label = 'coo'
coo_regex = ['operat', 'coo', 'ops']
coo_antiregex = ['africa', 'engin']

# CFO
cfo_label = 'cfo'
cfo_regex = ['fin', 'cfo', 'treas', 'reven', 'cfa', 'tres']  # sic
cfo_antiregex = []

# BD/partnerships
bd_label = 'bd'
bd_regex = ['partner', 'business', 'bus\.', 'growth', 'cpo',
            'biz', 'sales', 'ptr', 'p/s']
bd_antiregex = []

# president
pres_label = 'pres'
pres_regex = ['pres']
pres_antiregex = ['vice']

# marketing
cmo_label = 'cmo'
cmo_regex = ['marketing', 'cmo', 'pr ', 'communications']
cmo_antiregex = []

# development
cdo_label = 'cdo'
cdo_regex = ['development', 'cdo']
cdo_antiregex = ['bus', 'software', 'product', 'curriculum', 'global']

# board
board_label = 'board'
board_regex = ['board', 'chair', 'secretary', 'director$', 'trustee']
board_antiregex = ['project', 'program', 'exec', 'deputy', 'academic',
                   'engin', 'site', 'base', 'new york', 'sales', 'editorial',
                  'progam',  # sic
                  ]

# founder
founder_label = 'founder'
founder_regex = ['founder']
founder_antiregex = []

# list of all labels
labels = [ceo_label, cto_label, coo_label, cfo_label, bd_label,
          pres_label, cmo_label, cdo_label, board_label, founder_label]

In [32]:
# set titles: 1 if match, 0 if not
for label, regex_list, antiregex_list in [
    (ceo_label, ceo_regex, ceo_antiregex),
    (cto_label, cto_regex, cto_antiregex),
    (coo_label, coo_regex, coo_antiregex),
    (cfo_label, cfo_regex, cfo_antiregex),
    (bd_label, bd_regex, bd_antiregex),
    (pres_label, pres_regex, pres_antiregex),
    (cmo_label, cmo_regex, cmo_antiregex),
    (cdo_label, cdo_regex, cdo_antiregex),
    (board_label, board_regex, board_antiregex),
    (founder_label, founder_regex, founder_antiregex),
]:
    # column name is_label
    colname = 'is_%s' % label

    # default is not the title
    df[colname] = 0
    
    # if matches good regex, it is the title
    for regex in regex_list:
        idx = df['title'].str.contains(regex, case=False)
        df[colname].loc[idx] = 1
        
    # if matches anti-regex, override and make it not the title
    for regex in antiregex_list:
        idx = df['title'].str.contains(regex, case=False)
        df[colname].loc[idx] = 0

    # log
    rows = df[df[colname] == 1]
    print 'for', label, 'found', len(rows), 'rows at', len(rows['ein'].unique()), 'orgs with', len(rows['title'].unique()), 'titles'
    # print rows['title'].unique()

for ceo found 241 rows at 105 orgs with 69 titles
for cto found 129 rows at 34 orgs with 69 titles
for coo found 74 rows at 37 orgs with 29 titles
for cfo found 250 rows at 104 orgs with 66 titles
for bd found 48 rows at 19 orgs with 33 titles
for pres found 183 rows at 87 orgs with 45 titles
for cmo found 13 rows at 9 orgs with 10 titles
for cdo found 12 rows at 9 orgs with 8 titles
for board found 2027 rows at 137 orgs with 131 titles
for founder found 39 rows at 16 orgs with 22 titles


In [33]:
# number of titles held by this individual
df['n_indiv_titles'] = df[['is_%s' % label for label in labels]].sum(axis=1)
print df['n_indiv_titles'].value_counts()

1    2678
0     304
2     166
3       2
Name: n_indiv_titles, dtype: int64


In [34]:
# log uncategorized
uncategorized_rows = df[df['n_indiv_titles'] == 0]
print 'for uncategorized found', len(uncategorized_rows), 'rows at', len(uncategorized_rows['ein'].unique()), 'orgs with', len(uncategorized_rows['title'].unique()), 'titles'
# print uncategorized_rows['title'].unique()

for uncategorized found 304 rows at 71 orgs with 162 titles


In [35]:
# set up features for what's happening across the org in that year
for group_tuple, group in df.groupby(['org_name', 'tax_year']):
    print group_tuple
    for label in labels:
        # number of people with this title
        n_title_in_org_year = group['is_%s' % label].sum()
        
        # set number on df
        df.loc[group.index, 'n_%s_in_org_year' % label] = n_title_in_org_year

        # set boolean for whether title is present (0/1)
        df.loc[group.index, 'has_%s_in_org_year' % label] = 1 if n_title_in_org_year > 0 else 0
        
        # total compensation for people with this title
        total_comp_for_title = group[group['is_%s' % label] == 1]['comp'].sum()
        
        # set compensation on df
        df.loc[group.index, 'total_comp_%s_in_org_year' % label] = total_comp_for_title

        # set boolean for whether title is compensated (0/1)
        df.loc[group.index, 'has_comp_%s_in_org_year' % label] = 1 if total_comp_for_title > 0 else 0

(u'ADOPT-A-CLASSROOM INC', u'2010')
(u'ADOPT-A-CLASSROOM INC', u'2011')
(u'ADOPT-A-CLASSROOM INC', u'2012')
(u'ADOPT-A-CLASSROOM INC', u'2013')
(u'ADOPT-A-CLASSROOM INC', u'2014')
(u'Anjna Patient Education', u'2013')
(u'BENEFICENT TECHNOLOGY INC DBA Benetech', u'2014')
(u'BRACKETS FOR GOOD INC', u'2014')
(u'Blue Planet Network', u'2011')
(u'Blue Planet Network', u'2012')
(u'CANCER COMMONS', u'2012')
(u'CANCER COMMONS', u'2013')
(u'CANCER COMMONS', u'2014')
(u'CAREERVILLAGE INC', u'2012')
(u'CAREERVILLAGE INC', u'2013')
(u'CAREERVILLAGE INC', u'2014')
(u'CASE COMMONS INC', u'2012')
(u'CASE COMMONS INC', u'2013')
(u'CASE COMMONS INC', u'2014')
(u'CENTER FOR STUDENT OPPORTUNITY INC', u'2010')
(u'CENTER FOR STUDENT OPPORTUNITY INC', u'2011')
(u'CENTER FOR STUDENT OPPORTUNITY INC', u'2012')
(u'CENTER FOR STUDENT OPPORTUNITY INC', u'2013')
(u'CENTER FOR STUDENT OPPORTUNITY INC', u'2014')
(u'CLASSROOM INC', u'2010')
(u'CLASSROOM INC', u'2011')
(u'CLASSROOM INC', u'2012')
(u'CLASSROOM INC', u

In [36]:
# other features
df['org_age'] = pd.to_numeric(df['tax_year'], errors='coerce') - pd.to_numeric(df['year_formed'], errors='coerce')

In [37]:
# convert categorical features (city and state) to boolean
from sklearn import preprocessing

# integer code for cities
df['city'] = df['city'].str.upper()
cities = sorted(df['city'].unique())
df['city_int'] = df['city'].map(lambda x: cities.index(x))

# integer code for states
states = sorted(df['state'].unique())
df['state_int'] = df['state'].map(lambda x: states.index(x))

# perform one-hot enconding
enc = preprocessing.OneHotEncoder()
enc.fit(df[['city_int', 'state_int']])
transformed = enc.transform(df[['city_int', 'state_int']]).toarray()

# add features to df
header = ['in_%s' % city for city in cities] + ['in_%s' % state for state in states]
transformed_df = pd.DataFrame(transformed, columns=header)
df = pd.concat([df, transformed_df], axis=1)
print 'have features', df.columns.values

have features ['org_name' 'city' 'state' 'ein' 'url' 'tax_year' 'year_formed'
 'submission_time' 'n_employees' 'total_revenue' 'title' 'comp' 'is_ceo'
 'is_cto' 'is_coo' 'is_cfo' 'is_bd' 'is_pres' 'is_cmo' 'is_cdo' 'is_board'
 'is_founder' 'n_indiv_titles' 'n_ceo_in_org_year' 'has_ceo_in_org_year'
 'total_comp_ceo_in_org_year' 'has_comp_ceo_in_org_year'
 'n_cto_in_org_year' 'has_cto_in_org_year' 'total_comp_cto_in_org_year'
 'has_comp_cto_in_org_year' 'n_coo_in_org_year' 'has_coo_in_org_year'
 'total_comp_coo_in_org_year' 'has_comp_coo_in_org_year'
 'n_cfo_in_org_year' 'has_cfo_in_org_year' 'total_comp_cfo_in_org_year'
 'has_comp_cfo_in_org_year' 'n_bd_in_org_year' 'has_bd_in_org_year'
 'total_comp_bd_in_org_year' 'has_comp_bd_in_org_year' 'n_pres_in_org_year'
 'has_pres_in_org_year' 'total_comp_pres_in_org_year'
 'has_comp_pres_in_org_year' 'n_cmo_in_org_year' 'has_cmo_in_org_year'
 'total_comp_cmo_in_org_year' 'has_comp_cmo_in_org_year'
 'n_cdo_in_org_year' 'has_cdo_in_org_year' 'tot

In [38]:
# output
df.to_csv('output/comp_data_features.csv', index=False)